#### 붓꽃 품종 분류
- 목표: 붗꽃의 3개 품종을 분류하기
- 데이터셋 : 내장 데이터셋 
- 피쳐 : 4개
- 타겟 : 품종 1개
- 학습 : 지도학습 > 분류

[1] 데이터 준비

In [78]:
# 모듈 로딩
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [79]:
# 내장 데이터셋 로딩
data  = load_iris(as_frame=True)

In [80]:
# Bunch 인스턴스 => dict와 유사한 형태
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [81]:
featureDF = data['data']
targetSR = data['target']

In [82]:
featureDF.shape, targetSR.shape

((150, 4), (150,))

[2] 학습을 위한 데이터 셋 준비 => 학습용,검증용,테스트용

In [83]:
# 학습용 & 테스트용 분리
X_train,X_test,y_train,y_test = train_test_split(featureDF,
                                                 targetSR,
                                                 stratify = targetSR)

In [84]:
# 학습용 & 검증용 분리
X_train,X_val,y_train,y_val = train_test_split(X_train,
                                                 y_train,
                                                 stratify = y_train)

In [85]:
print(f'Train DS : {X_train.shape[0]} {X_train.shape[0]/featureDF.shape[0]}%')
print(f'val DS : {X_val.shape[0]} {X_val.shape[0]/featureDF.shape[0]:.2f}%')
print(f'test DS : {X_test.shape[0]} {X_test.shape[0]/featureDF.shape[0]:.2f}%')

Train DS : 84 0.56%
val DS : 28 0.19%
test DS : 38 0.25%


[3] 교차검증 방식

In [86]:
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

In [87]:
# 모델 인스턴스 생성
dtc_model = DecisionTreeClassifier()

#[3-1] Kfold 기반 ----------
# 정확도 저장 리스트
accuracys = []

# Kfold 인스턴스 생성 [기본 k=5]
kfold = KFold()

In [88]:
# k번 만큼 학습 진행
# -> k 등분 후 학습용 데이터셋 인덱스, 검증용 데이터셋 인덱스
for idx,(train_index,val_index) in enumerate(kfold.split(featureDF),1):

    # print(f'train_index: {train_index.tolist()}')

    # X_train,X_val 데이터셋 설정
    X_train,y_train = featureDF.iloc[train_index.tolist()],targetSR[train_index.tolist()]
    X_val,y_val = featureDF.iloc[val_index.tolist()],targetSR[val_index.tolist()]

    # 학습 진행
    dtc_model.fit(X_train,y_train)

    # 평가 => 분류의 경우 score()메서드 -> 정확도 반환
    train_acc = dtc_model.score(X_train,y_train)
    val_acc = dtc_model.score(X_val,y_val)

    accuracys.append([train_acc,val_acc])
    print(f'[{idx}번째] Train 정확도 : {train_acc}    Val 정확도 : {val_acc}')

[1번째] Train 정확도 : 1.0    Val 정확도 : 1.0
[2번째] Train 정확도 : 1.0    Val 정확도 : 1.0
[3번째] Train 정확도 : 1.0    Val 정확도 : 0.8666666666666667
[4번째] Train 정확도 : 1.0    Val 정확도 : 0.9333333333333333
[5번째] Train 정확도 : 1.0    Val 정확도 : 0.8


In [89]:
accuracys

[[1.0, 1.0],
 [1.0, 1.0],
 [1.0, 0.8666666666666667],
 [1.0, 0.9333333333333333],
 [1.0, 0.8]]

In [90]:
kfold.n_splits

5

In [91]:
# 평균 계산
train_mean = sum([value[0] for value in accuracys])/kfold.n_splits
test_mean = sum([value[1] for value in accuracys])/kfold.n_splits

In [92]:
print(f'Train 정확도 : {train_mean}   val 정확도 : {test_mean:.2f}')

Train 정확도 : 1.0   val 정확도 : 0.92


In [93]:
#### ===> [3-2] StratifiedKFold : 정답 /레이블 / 타겟의 데이터 비율 고려해서 데이터 나눔

accuracys = []

# Kfold 인스턴스 생성 [기본 k=5]
skfold = StratifiedKFold()


# k번 만큼 k개 데이터셋으로 학습 진행
# -> k 등분 후 학습용 데이터셋 인덱스, 검증용 데이터셋 인덱스
for idx,(train_index,val_index) in enumerate(skfold.split(featureDF,targetSR),1):

    # print(f'train_index: {train_index.tolist()}')

    # X_train,X_val 데이터셋 설정
    X_train,y_train = featureDF.iloc[train_index.tolist()],targetSR[train_index.tolist()]
    X_val,y_val = featureDF.iloc[val_index.tolist()],targetSR[val_index.tolist()]

    # 학습 진행
    dtc_model.fit(X_train,y_train)

    # 평가 => 분류의 경우 score()메서드 -> 정확도 반환
    train_acc = dtc_model.score(X_train,y_train)
    val_acc = dtc_model.score(X_val,y_val)

    accuracys.append([train_acc,val_acc])
    print(f'[{idx}번째] Train 정확도 : {train_acc}    Val 정확도 : {val_acc}')

[1번째] Train 정확도 : 1.0    Val 정확도 : 0.9666666666666667
[2번째] Train 정확도 : 1.0    Val 정확도 : 0.9666666666666667
[3번째] Train 정확도 : 1.0    Val 정확도 : 0.9
[4번째] Train 정확도 : 1.0    Val 정확도 : 1.0
[5번째] Train 정확도 : 1.0    Val 정확도 : 1.0


In [94]:
# 평균 계산
train_mean = sum([value[0] for value in accuracys])/skfold.n_splits
test_mean = sum([value[1] for value in accuracys])/skfold.n_splits

In [95]:
print(f'Train 정확도 : {train_mean}   val 정확도 : {test_mean:.2f}')        

Train 정확도 : 1.0   val 정확도 : 0.97


- 교차검증 및 성능평가 동시 진행 함수 
* => cross_val_score,cross_val_predict
* => cross_validate

In [96]:
from sklearn.model_selection import cross_val_predict,cross_val_score,cross_validate

In [97]:
### 전체 DS => 학습용과 테스트용 DS 분리
X_train,X_test,y_train,y_test = train_test_split(featureDF,
                                                 targetSR,
                                                 stratify = targetSR)

In [98]:
#### cross_val_predicts
precict = cross_val_predict(dtc_model,X_train,y_train,cv=3)

In [99]:
print(f'precict : {precict}')

precict : [0 2 0 2 2 0 0 2 2 0 2 1 0 0 1 0 2 2 0 0 0 1 0 2 2 0 2 2 1 0 2 2 2 1 0 2 2
 0 1 0 2 1 0 0 2 0 0 2 2 0 1 2 1 0 2 1 2 0 1 1 2 1 2 0 1 2 1 1 0 1 2 0 1 2
 2 1 0 2 1 1 2 2 1 2 2 1 0 2 0 1 0 0 2 1 2 1 0 1 1 0 1 0 1 0 0 2 2 1 1 1 1
 1]


In [100]:
#### cross_val_score
precict = cross_val_score(dtc_model,X_train,y_train,cv=3)

In [101]:
print(f'precict : {precict}')

precict : [0.92105263 0.94594595 0.94594595]


In [102]:
### cross_validate
result = cross_validate(dtc_model,X_train,y_train,
                        return_train_score=True,
                        return_estimator = True)

In [103]:
result

{'fit_time': array([0.        , 0.        , 0.00200081, 0.00099587, 0.00099659]),
 'score_time': array([0.        , 0.0084362 , 0.00098848, 0.00099611, 0.00099015]),
 'estimator': [DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier()],
 'test_score': array([0.91304348, 0.91304348, 0.95454545, 0.95454545, 0.95454545]),
 'train_score': array([1., 1., 1., 1., 1.])}

In [104]:
resultDF = pd.DataFrame(result).loc[:,['test_score','train_score']]

In [105]:
resultDF

,test_score,train_score
0,0.913043,1.0
1,0.913043,1.0
2,0.954545,1.0
3,0.954545,1.0
4,0.954545,1.0


In [107]:
# 가장 성능이 좋은 모델 뽑기 
best_model = result['estimator'][2]

In [108]:
best_model.predict(X_test)

array([2, 2, 0, 1, 2, 2, 0, 1, 1, 1, 2, 2, 0, 0, 1, 2, 0, 1, 0, 0, 2, 1,
       0, 2, 1, 2, 1, 1, 2, 2, 1, 0, 0, 0, 0, 1, 2, 0])

In [109]:
best_model.score(X_test,y_test)

1.0